In [2]:
import cv2
import numpy as np
import os
import tensorflow as tf

In [3]:
class CaptionGenerator:
    
    IMAGE_WIDTH = 0
    IMAGE_HEIGHT = 0
    
    IMAGE_CLASSIFIER = None
    
    def __init__(self, image_width, image_height):
        self.IMAGE_WIDTH = image_width
        self.IMAGE_HEIGHT = image_height
        
        self.IMAGE_CLASSIFIER = ImageClassifier(image_width, image_height)
        self.IMAGE_CLASSIFIER.loadModel()
        
    def convertPath(self, path):
        validPath = ""
        for char in path:
            if char == '\\':
                validPath += "/"
            else:
                validPath += char
        return validPath
    
    def generateCaption(self, image_path, prediction_model_path="ImageClassifier/ImageClassifier.h5"):
        image_path = self.convertPath(image_path)
        image = self.readImageFile(image_path)
        self.IMAGE_CLASSIFIER.predict(image)
    
    def readImageFile(self, image_path):
        if not os.path.isfile(image_path):
            print("Unknow path")
            return None
        else:
            image = cv2.imread(image_path)
            image = cv2.resize(image, (self.IMAGE_WIDTH, self.IMAGE_HEIGHT))
            image = np.expand_dims(image, axis=0)
            return image

In [4]:
class ImageClassifier:
    
    DATASETS_PATH = "D:/Cours/Cesi/A5/UE/Option - Data Science/Projet/Livrable 2/Datasets/extracted"
    CLASS_NAMES = ['Painting', 'Photo', 'Schematics', 'Sketch', 'Text']
    
    VALIDATION_SPLIT = 0.3
    
    IMAGE_WIDTH = 0
    IMAGE_HEIGHT = 0
    
    MODEL = None
    EPOCHS = 0
    HISTORY = None
    
    def __init__(self, image_width, image_height):
        self.IMAGE_WIDTH = image_width
        self.IMAGE_HEIGHT = image_height
        
    def buildModel(self, dropout_rate=0, kernel_regularizer_l1=0.00, kernel_regularizer_l2=0.0):
        model = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(self.IMAGE_HEIGHT, self.IMAGE_WIDTH, 3)),
            tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(self.IMAGE_HEIGHT, self.IMAGE_WIDTH, 3)),
            tf.keras.layers.experimental.preprocessing.RandomRotation(10),
            tf.keras.layers.experimental.preprocessing.RandomZoom((0.2, 0.5)),
            tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu", 
                                   kernel_regularizer=tf.keras.regularizers.l1_l2(l1=kernel_regularizer_l1, l2=kernel_regularizer_l2)),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu", 
                                   kernel_regularizer=tf.keras.regularizers.l1_l2(l1=kernel_regularizer_l1, l2=kernel_regularizer_l2)),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu", 
                                   kernel_regularizer=tf.keras.regularizers.l1_l2(l1=kernel_regularizer_l1, l2=kernel_regularizer_l2)),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l1_l2(l1=kernel_regularizer_l1, l2=kernel_regularizer_l2)),
            tf.keras.layers.Dense(len(self.CLASS_NAMES))
        ])
        
        model.compile(optimizer="adam",
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=["accuracy"])
        return model
        
    def fit(self, epochs, save_path=None, show_training_results=False, dropout_rate=0, kernel_regularizer_l1=0.01, kernel_regularizer_l2=0.01):
        train_dataset, test_dataset = self.generateDatasets()
        self.MODEL = self.buildModel(dropout_rate, kernel_regularizer_l1, kernel_regularizer_l2)
        self.EPOCHS = epochs
        
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
        self.HISTORY = self.MODEL.fit(train_dataset, validation_data=test_dataset, epochs=epochs, callbacks=[callback])
        
        if save_path != None:
            self.MODEL.save(save_path)
            
        if show_training_results:
            self.showTrainingResults()
    
    def generateDatasets(self):
        datasets = []
        
        for subset_label in ['training', 'validation']:
            datasets.append(tf.keras.preprocessing.image_dataset_from_directory(
            self.DATASETS_PATH,
            labels="inferred",
            label_mode="int",
            validation_split=self.VALIDATION_SPLIT,
            subset=subset_label,
            seed=42,
            color_mode="rgb",
            image_size=(self.IMAGE_WIDTH, self.IMAGE_HEIGHT)))
        return datasets[0], datasets[1]
    
    def loadModel(self, model_path="ImageClassifier/ImageClassifier.h5", add_softmax_layer=True):
        model = tf.keras.models.load_model(model_path)
        if add_softmax_layer:
            model.add(tf.keras.layers.Softmax())
        self.MODEL = model
        
    def modelSummary(self):
        self.MODEL.summary()
        
    def predict(self, image, model_path="ImageClassifier/ImageClassifier.h5"):
        prediction = self.MODEL.predict(tf.convert_to_tensor(image))
        print(self.CLASS_NAMES[np.argmax(prediction)])
        
    def showTrainingResults():
        epochs_range = range(self.EPOCHS)
    
        plt.figure(figsize=(8, 8))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, self.HISTORY.history['accuracy'], label="Training accuracy")
        plt.plot(epochs_range, self.HISTORY.history['val_accuracy'], label="Validation accuracy")
        plt.legend()
        plt.title("Training and validation accuracy")

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, self.HISTORY.history['loss'], label="Training loss")
        plt.plot(epochs_range, self.HISTORY.history['val_loss'], label="Validation loss")
        plt.legend()
        plt.title("Training and validation loss")

        plt.show()

In [5]:
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256

In [6]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier1.h5", show_training_results=True, 
                     dropout_rate=0, kernel_regularizer_l1=0.0, kernel_regularizer_l2=0.0)

Found 49024 files belonging to 5 classes.
Using 34317 files for training.
Found 49024 files belonging to 5 classes.
Using 14707 files for validation.
Epoch 1/30
  10/1073 [..............................] - ETA: 31:41 - loss: 6.1460 - accuracy: 0.3042

KeyboardInterrupt: 

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier2.h5", show_training_results=True, 
                     dropout_rate=0.2, kernel_regularizer_l1=0.0, kernel_regularizer_l2=0.0)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier3.h5", show_training_results=True, 
                     dropout_rate=0.5, kernel_regularizer_l1=0.0, kernel_regularizer_l2=0.0)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier4.h5", show_training_results=True, 
                     dropout_rate=0.8, kernel_regularizer_l1=0.0, kernel_regularizer_l2=0.0)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier5.h5", show_training_results=True, 
                     dropout_rate=0.5, kernel_regularizer_l1=0.01, kernel_regularizer_l2=0.0)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier6.h5", show_training_results=True, 
                     dropout_rate=0.5, kernel_regularizer_l1=0.001, kernel_regularizer_l2=0.0)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier7.h5", show_training_results=True, 
                     dropout_rate=0.2, kernel_regularizer_l1=0.0, kernel_regularizer_l2=0.01)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier8.h5", show_training_results=True, 
                     dropout_rate=0.5, kernel_regularizer_l1=0.0, kernel_regularizer_l2=0.001)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier9.h5", show_training_results=True, 
                     dropout_rate=0.5, kernel_regularizer_l1=0.01, kernel_regularizer_l2=0.01)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier10.h5", show_training_results=True, 
                     dropout_rate=0.5, kernel_regularizer_l1=0.001, kernel_regularizer_l2=0.001)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier11.h5", show_training_results=True, 
                     dropout_rate=0.8, kernel_regularizer_l1=0.001, kernel_regularizer_l2=0.001)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier12.h5", show_training_results=True, 
                     dropout_rate=0.8, kernel_regularizer_l1=0.01, kernel_regularizer_l2=0.01)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier13.h5", show_training_results=True, 
                     dropout_rate=0.2, kernel_regularizer_l1=0.01, kernel_regularizer_l2=0.01)

In [ ]:
image_classifier = ImageClassifier(IMAGE_WIDTH, IMAGE_HEIGHT)
image_classifier.fit(epochs=30, save_path="ImageClassifier/ImageClassifier14.h5", show_training_results=True, 
                     dropout_rate=0.2, kernel_regularizer_l1=0.001, kernel_regularizer_l2=0.001)

In [ ]:
#captionGenerator = CaptionGenerator(IMAGE_WIDTH, IMAGE_HEIGHT)
#captionGenerator.generateCaption("D:/Cours/Cesi/A5/UE/Option - Data Science/Projet/Dataset soutenance/Text  (4).png")
#captionGenerator.generateCaption("D:/Cours/Cesi/A5/UE/Option - Data Science/Projet/Dataset soutenance/Paint  (1).jpg", 
#                                 prediction_model_path="ImageClassifier/ImageClassifierFullPainting.h5")